In [179]:
import numpy as np
from hmmlearn import hmm

In [180]:
data = np.load('sequences.npy')
data.shape

(200, 30)

In [181]:
np.unique(data)

array([1, 2, 3, 4, 5, 6])

In [182]:
X = data.reshape(200*30, 1)
X.shape

(6000, 1)

In [183]:
lens = np.ones(data.shape[0])*30
lens = lens.astype(int)
np.sum(lens)

6000

In [184]:
model = hmm.CategoricalHMM(n_components=2, random_state=10)
model

CategoricalHMM(n_components=2, random_state=10)

In [185]:
model.fit(X, lens)

CategoricalHMM(n_components=2,
               random_state=RandomState(MT19937) at 0x1FA26E30B40)

In [186]:
model.score(X)

-10434.902086730863

In [187]:
np.round(model.startprob_, 3)

array([0.618, 0.382])

In [188]:
np.round(model.transmat_, 3)

array([[0.888, 0.112],
       [0.156, 0.844]])

In [189]:
model.emissionprob_

array([[0.        , 0.15798992, 0.16388603, 0.18417735, 0.1710482 ,
        0.19124892, 0.13164957],
       [0.        , 0.1200444 , 0.09791682, 0.09568956, 0.10791744,
        0.08764206, 0.49078972]])

In [190]:
# for table
for e in model.emissionprob_:
    strings = ["%.3f" % i for i in e]
    p = ''
    for i in strings:
        p += i +' & '
    print(p)

0.000 & 0.158 & 0.164 & 0.184 & 0.171 & 0.191 & 0.132 & 
0.000 & 0.120 & 0.098 & 0.096 & 0.108 & 0.088 & 0.491 & 


In [191]:
# forward algorithm
iprob = model.startprob_
tprob = model.transmat_
eprob = model.emissionprob_

for t in range(4):
    # as = np.zeros([2, 2])
    if t==0:
        a0 = eprob[0, 6]*iprob[0]
        a1 = eprob[1, 6]*iprob[1]
    else:
        a0 = eprob[0, 6]*(a0*tprob[0, 0] + a1*tprob[1, 0])
        a1 = eprob[1, 6]*(a0*tprob[0, 1] + a1*tprob[1, 1])
p = a0 + a1
p

0.014626307201743518

In [192]:
# backward algorithm
iprob = model.startprob_
tprob = model.transmat_
eprob = model.emissionprob_

for t in [3, 2, 1, 0]:
    # as = np.zeros([2, 2])
    if t==3:
        b0 = 1
        b1 = 1
    else:
        b0 = tprob[0, 0]*eprob[0, 6]*b0 + tprob[0, 1]*eprob[1, 6]*b1
        b1 = tprob[1, 0]*eprob[0, 6]*b0 + tprob[1, 1]*eprob[1, 6]*b1
p = a0 + a1
p

0.014626307201743518

In [193]:
seq = np.array([3, 2, 1, 3, 4, 5, 6, 3, 1, 4, 1, 6, 6, 2, 6])
seq = seq.reshape(1, -1)
model.decode(seq)

(-28.45720629383466,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], dtype=int64))